In [3]:
from pyspark.sql.types import *
from operator import add
from datetime import datetime

print('Added required libraries')


adlpath_dir = r'adl://merckadl.azuredatalakestore.net/test_file_write/'
file_system_base_uri = "adl://merckadl.azuredatalakestore.net"
adlpath = r'adl://merckadl.azuredatalakestore.net/test_file_write/result.txt'
file01 = 'adl://merckadl.azuredatalakestore.net/extracted/pdb1u10.txt'
source_folder_localtion = '/extracted/'

print('Declared various required path locations')


URI           = sc._gateway.jvm.java.net.URI
Path          = sc._gateway.jvm.org.apache.hadoop.fs.Path
FileSystem    = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
Configuration = sc._gateway.jvm.org.apache.hadoop.conf.Configuration
print('Got JVM flags')

fs = FileSystem.get(URI(file_system_base_uri), Configuration())
status = fs.listStatus(Path(source_folder_localtion))

all_files_found = []
for fileStatus in status:
    all_files_found.append(str(status[0].getPath()))

print('Found '+str(len(all_files_found))+' files in data lake store')

print('Got file list from Azure data lake storage using JAVA class')
#for fileStatus in status:
#    print(str(status[0].getPath()))
    
    
print('Finding repeating words in all files')    
t1 = datetime.now()
lines = spark.read.text(all_files_found).rdd.map(lambda r: r[0])
counts = lines.flatMap(lambda x: x.split(' ')) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add)
output = counts.collect()
t2 = datetime.now()
delta = t2 - t1
print('Counting repeating words from PDB files')
repeated_word_count = 0
for (word, count) in output:
    if count > 1:
        repeated_word_count+=1
print('\nTotal '+str(repeated_word_count)+' repeating words found in PDB files\n')

print('Saving the data for future analysis')
counts.saveAsTextFile(adlpath)

Added required libraries
Declared various required path locations
Got JVM flags
Found 135 files in data lake store
Got file list from Azure data lake storage using JAVA class
Finding repeating words in all files
Counting repeating words from PDB files

Total 45063 repeating words found in PDB files

Saving the data for future analysis